<a href="https://colab.research.google.com/github/flickmatch/home/blob/Shivam_Model-ML/Goal_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Binary classification Model for Goal Detection

Importing libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')

splitting input data into train and validation set

In [ ]:
def get_dataset(
    batch_size=32,
    img_height=224,
    img_width=224,
    data_dir='',                  # path to directory where training data is available
    validation_split=0.2,
    test_data_dir=''              # path to directory where test data is available
):
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=validation_split,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    validation_dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=validation_split,
        subset="validation",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_data_dir,
        validation_split=None,  # No validation split for test dataset
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    class_names = train_dataset.class_names
    return train_dataset, validation_dataset, test_dataset, class_names

Binary Classification training script based on CNNs

In [ ]:
def get_vgg16_model():
  # VGG16
  vgg = tf.keras.applications.VGG16(
      include_top=False,
      weights='imagenet',
      input_shape=(224, 224, 3),
      pooling='avg'
  )
  vgg.trainable = False
  return vgg


def get_resnet50_model():
  # Resnet50
  resnet = tf.keras.applications.resnet50.ResNet50(
      include_top=False,
      weights='imagenet',
      input_shape=(224, 224, 3),
      pooling='avg'
  )
  resnet.trainable = False
  return resnet


def get_model(
  checkpoint_path,
  model_name,
  load_weights=False
):
  epoch_path = os.path.join(checkpoint_path, 'cp-{epoch:04d}.ckpt')
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the model's weights every 5 epochs
  cp_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_path,
      verbose=1,
      save_weights_only=True,
      save_freq='epoch')

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Rescaling(1./255))

  if model_name == 'vgg16':
    model.add(get_vgg16_model())
  elif model_name == 'resnet50':
    model.add(get_resnet50_model())
  else:
    raise ValueError(f'Provided model_name={model_name} is not supported today.')

  model.add(tf.keras.layers.Dense(256, activation='relu'))
  model.add(tf.keras.layers.Dense(1))

  model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

  if load_weights:
    model.load_weights(checkpoint_path)

  return model, cp_callback

In [ ]:
def train(
    train_dataset,
    validation_dataset,
    model,
    cp_callback,
    epochs=3
):
  history = model.fit(
      train_dataset,
      validation_data=validation_dataset,
      epochs=epochs,
      callbacks=[cp_callback]
  )
  return history

Visualize result and save model

In [ ]:
def visualize_results(
    history,
    epochs
):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(epochs)

  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()

In [ ]:
def save_model(
    model,
    save_path
):
  model.save(save_path)

Main Training loop

In [ ]:
def main():
    train_dataset, validation_dataset, test_dataset, class_names = get_dataset(
        batch_size=32,
        img_height=224,
        img_width=224,
        data_dir='',                    # path to your training data  directory
        validation_split=0.2,
        test_data_dir=''                # Path to your test data directory
    )

    model, cp_callback = get_model(
        checkpoint_path='',             # path to checkpoint
        model_name='vgg16',
        load_weights=False
    )

    input_shape = (32, 224, 224, 3)
    model.build(input_shape)
    print(model.summary())

    epochs = 20
    history = train(
        train_dataset,
        validation_dataset,
        model,
        cp_callback,
        epochs=epochs
    )

    visualize_results(
        history,
        epochs
    )

    save_model(model, save_path='')                              # provide path to save the model at

    if test_dataset:
        # Evaluate the model on the test dataset
        test_loss, test_accuracy = model.evaluate(test_dataset)
        print("Test Loss:", test_loss)
        print("Test Accuracy:", test_accuracy)

main()
